# Dataframe for Toronto Neighborhoods

Import the libraries we need.

In [4]:
!pip install geocoder

     |████████████████████████████████| 102kB 11.4MB/s ta 0:00:01


In [3]:
!pip install folium

     |████████████████████████████████| 102kB 10.2MB/s ta 0:00:01


In [1]:
!pip install geopy
from geopy.geocoders import Nominatim 

In [5]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 

## PART 1: LOADING DF

Scrape the content from the wikipedia website. 

In [6]:
html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=urllib.request.urlopen(html).read()
soup=BeautifulSoup(source, 'html.parser')

Make the html content into a table.

In [7]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = df[0]

Remove not assigned values.

In [8]:
df = df[df["Borough"] != "Not assigned"]

Merge those values that have the same Postal Code.

In [9]:
df = df.groupby(['Postal Code', "Borough"])["Neighborhood"].apply(lambda x: ','.join(x)).reset_index()
df.shape

(103, 3)

## PART 2: Add coordinates etc

In [10]:
import geocoder 

The geocoder is not working, but I am pretty sure my code is fine. There is no error message, the code just keeps running forever....

In [11]:
# initialize your variable to None
lat_lng_coords = None
latitude = dict()
longitude = dict()

for postal_code in df["Postal Code"]:
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.update({postal_code:lat_lng_coords[0]})
    longitude.update({postal_code:lat_lng_coords[1]})


KeyboardInterrupt: 

So I add the data manually from the csv.

In [12]:
df = df.merge(pd.read_csv("http://cocl.us/Geospatial_data"),"left", left_on = "Postal Code", right_on = "Postal Code")

In [13]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Now, let's explore the neighborhoods.

Frst, I find the location for Toronto.

In [14]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Then I map Toronto and add all the neighborhoods

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We can see that the neighborhoods cluster tighter together at the coastline, what seems to be the center of Toronto and then spread out from there.

In [16]:
#@hidden_cell
CLIENT_ID = 'AFDQKCBOR1KIQEODCJGW15J0WL2W40FNX34FM3GFGARFVC2L' # your Foursquare ID
CLIENT_SECRET = '22NUORW5AHPQIEIJRQND30UXI1TIUUD14JL1VKN0DEF3X3GG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

Now, let's check out some neighborhoods. Let's take the first neighborhood: Malvern, Rouge. This can be done with any of them.

In [17]:
df.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [18]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
import requests
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items)

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(15)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Downtown Toronto,Neighborhood,NaN,CA,Toronto,Canada,NaN,113,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,5227bb01498e17bf485e6202
1,Nathan Phillips Square,Plaza,100 Queen St W,CA,Toronto,Canada,at Bay St,138,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65227047322295...",43.652270,-79.383516,NaN,M5H 2N1,ON,4ad4c05ef964a520a6f620e3
2,Chatime 日出茶太,Bubble Tea Shop,132 Dundas St W,CA,Toronto,Canada,btwn Bay & University,237,"[132 Dundas St W (btwn Bay & University), Toro...","[{'label': 'display', 'lat': 43.65554164147378...",43.655542,-79.384684,NaN,M5G 1C3,ON,4e2284b11fc7c0ef9857d143
3,Textile Museum of Canada,Art Museum,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,230,[55 Centre Avenue (University Ave. and Dundas ...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,NaN,M5G 2H5,ON,4ad4c05ef964a520e2f620e3
4,Indigo,Bookstore,220 Yonge St,CA,Toronto,Canada,NaN,260,"[220 Yonge St, Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.65351471121164...",43.653515,-79.380696,Downtown Yonge,M5B 2H1,ON,4b2a6eb8f964a52012a924e3
5,Japango,Sushi Restaurant,122 Elizabeth St.,CA,Toronto,Canada,at Dundas St. W,222,"[122 Elizabeth St. (at Dundas St. W), Toronto ...","[{'label': 'display', 'lat': 43.65526771691681...",43.655268,-79.385165,NaN,M5G 1P5,ON,4ae7b27df964a52068ad21e3
6,LUSH,Cosmetics Shop,"220 Yonge St, Unit B215-A",CA,Toronto,Canada,in Toronto Eaton Centre,284,"[220 Yonge St, Unit B215-A (in Toronto Eaton C...","[{'label': 'display', 'lat': 43.653557, 'lng':...",43.653557,-79.380400,NaN,M5B 2H1,ON,4bd0b30d41b9ef3b8fa0fae5
7,Poke Guys,Poke Place,112 Elizabeth St,CA,Toronto,Canada,at Dundas St W,181,"[112 Elizabeth St (at Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.65489527525682...",43.654895,-79.385052,NaN,M5G 1P5,ON,57bcd3b7498e652a678d0378
8,Old City Hall,Monument / Landmark,60 Queen Street West,CA,Toronto,Canada,NaN,240,"[60 Queen Street West, Toronto ON M5H 1A1, Can...","[{'label': 'display', 'lat': 43.65200880087612...",43.652009,-79.381744,NaN,M5H 1A1,ON,4ad4c05ef964a5208ef620e3
9,M Square Coffee Co,Coffee Shop,C24 - 123 Queen St W,CA,Toronto,Canada,NaN,253,"[C24 - 123 Queen St W, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65121797253777...",43.651218,-79.383555,NaN,NaN,ON,54132b3b498ee9ca9332e189


We can see there is quite the selection of venues in this neighborhood.
Let's check for Hotels.

In [21]:
dataframe_filtered[dataframe_filtered["categories"] == "Hotel"]

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
12,DoubleTree by Hilton,Hotel,108 Chestnut Street,CA,Toronto,Canada,Dundas St W,204,"[108 Chestnut Street (Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.6546083, 'lng'...",43.654608,-79.385942,NaN,M5G 1R3,ON,52ce14b0498e50457ce11780


There appears to be one hotel in this neighborhood. Let's map it.

In [22]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add hotel spot to the map as blue circle markers
lat = dataframe_filtered.loc[12, "lat"]
lng = dataframe_filtered.loc[12, "lng"]
label = dataframe_filtered.loc[12, "name"]
folium.CircleMarker(
    [lat, lng],
    radius=10,
    popup=label,
    fill=True,
    color='blue',
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Cool, it seems to be right at the center of our neighborhood. Let's see if there's any cool things to do around the hotel. I want to go shopping and to a museum. Let's check it out. 

In [23]:
df_shopping = dataframe_filtered.loc[dataframe_filtered["categories"].isin(["Shopping Center", "Art Museum", "Clothing Store"])]
df_shopping.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
3,Textile Museum of Canada,Art Museum,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,230,[55 Centre Avenue (University Ave. and Dundas ...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,NaN,M5G 2H5,ON,4ad4c05ef964a520e2f620e3
17,UNIQLO ユニクロ,Clothing Store,220 Yonge St,CA,Toronto,Canada,at Dundas St W,378,"[220 Yonge St (at Dundas St W), Toronto ON M5B...","[{'label': 'display', 'lat': 43.65591027779457...",43.655910,-79.380641,Downtown Toronto,M5B 2H1,ON,57eda381498ebe0e6ef40972
19,Nordstrom,Clothing Store,260 Yonge Street,CA,Toronto,Canada,NaN,295,"[260 Yonge Street, Toronto ON M5B 2L9, Canada]","[{'label': 'display', 'lat': 43.655041, 'lng':...",43.655041,-79.380966,NaN,M5B 2L9,ON,57d1e1a2498e96cce6f32217


In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add hotel spots to the map as blue circle markers
lat = dataframe_filtered.loc[12, "lat"]
lng = dataframe_filtered.loc[12, "lng"]
label = dataframe_filtered.loc[12, "name"]
folium.CircleMarker(
    [lat, lng],
    radius=10,
    popup=label,
    fill=True,
    color='blue',
    fill_color='blue',
    fill_opacity=0.6
    ).add_to(venues_map)

# add shopping spots to the map as blue circle markers
for lat, lng, label in zip(df_shopping.lat, df_shopping.lng, df_shopping.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

Perfect, looks like the museum is really close!

Let's see if these venues have good ratings.

In [25]:
for venue_id in df_shopping["id"]:

    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')

8.2
8.7
7.9


yay, they all look pretty ok.

## PART 3: CLUSTERING & ANALYSIS 

In [106]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    # make the GET request
results = requests.get(url).json()
for v in results["response"]["groups"][0]["items"]:
    print(v["venue"]["id"])

5273f6f311d2137549494e29
4dde924bd22d28453ec2f75c
4da741691e725f12297f3a53


In [107]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',"Venue ID",
                             'Venue Category']
    
    return(nearby_venues)

In [108]:
toronto_nearby = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


In [109]:
toronto_nearby

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,4bb6b9446edc76b0d771311c,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,4c23d3aaf7ced13a5ed7216d,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,4beee041e24d20a1cd857314,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,4c62f34bde1b2d7fec89e370,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,57fd24f6cd1083addfd77bf9,Restaurant
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,5411f741498e9ebd5e35d8bd,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,4c1c7f9bb306c9288f0464b7,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,4b9008acf964a520737133e3,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,513b84e6e4b0ef935943c0ad,Intersection
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,4d95bd2ddaec224bf7bf043e,Breakfast Spot


I wanted to calculate the average rating in each neighborhood, but I exceed by quota....

In [119]:
for venue_id in toronto_nearby["Venue ID"]:

    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    rating = []
    
    result = requests.get(url).json()
    try:
        rating = rating.append(result['response']['venue']['rating'])
    except:
        rating = rating.append(None)
        
toronto_nearby["rating"] = rating

So, instead, let's count venue types..

First, we use one-hot encoding to find the counts of all venues 

In [129]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_nearby[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_nearby['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = toronto_onehot.columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Then, we look at the 10 most common

In [180]:
import numpy as np

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Skating Rink,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Pub,Dance Studio,Sandwich Place,Coffee Shop,Gym,Gas Station,Coworking Space
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Frozen Yogurt Shop,Deli / Bodega,Middle Eastern Restaurant,Chinese Restaurant,Mobile Phone Shop,Supermarket,Fried Chicken Joint
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Department Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Pub


### Clustering

We cluster the data based on these most common venues

In [181]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [182]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(15) # check the last columns!



,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3.0,Bar,Yoga Studio,Farmers Market,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Medical Center,Intersection,Electronics Store,Restaurant,Rental Car Location,Bank,Mexican Restaurant,Breakfast Spot,Discount Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Pharmacy,Korean Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Diner,Dessert Shop
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1.0,Playground,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,3.0,Department Store,Coffee Shop,Hobby Shop,Chinese Restaurant,Bus Station,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,3.0,Bus Line,Bakery,Ice Cream Shop,Park,Metro Station,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3.0,American Restaurant,Motel,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,3.0,Café,College Stadium,General Entertainment,Skating Rink,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [158]:
toronto_merged.head(20) 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3.0,Bar,Yoga Studio,Farmers Market,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Medical Center,Intersection,Electronics Store,Restaurant,Rental Car Location,Bank,Mexican Restaurant,Breakfast Spot,Discount Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Pharmacy,Korean Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Diner,Dessert Shop
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1.0,Playground,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,3.0,Department Store,Coffee Shop,Hobby Shop,Chinese Restaurant,Bus Station,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,3.0,Bus Line,Bakery,Ice Cream Shop,Park,Metro Station,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3.0,American Restaurant,Motel,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,3.0,Café,College Stadium,General Entertainment,Skating Rink,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


I see that some of these vaues have a NaN value, so we drop the NaN values

In [159]:
toronto_merged = toronto_merged.dropna(subset = ["Cluster Labels"])

Then I map them

In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Looks like it clustered into 4 main categories, parks, playgrounds, amenities and food, baseball fields and a random spot somewhat outside... I think  the way it's clustered isn't ideal, since the amenities and food is way to broad. Maybe, next time, I should explore finding a better number of clusters.

In [166]:
parks = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
parks

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,0.0,Park,Convenience Store,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
25,North York,0.0,Park,Food & Drink Shop,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
40,East York,0.0,Park,Convenience Store,Metro Station,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
44,Central Toronto,0.0,Photography Studio,Park,Swim School,Bus Line,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
48,Central Toronto,0.0,Tennis Court,Park,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
50,Downtown Toronto,0.0,Park,Playground,Trail,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
64,Central Toronto,0.0,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
74,York,0.0,Park,Women's Store,Pool,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
79,North York,0.0,Construction & Landscaping,Park,Bakery,Trail,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
90,Etobicoke,0.0,Park,River,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


In [168]:
playgrounds = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
playgrounds

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1.0,Playground,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
14,Scarborough,1.0,Playground,Park,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


In [170]:
baseball_fields = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
baseball_fields 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,2.0,Baseball Field,Yoga Studio,Farmers Market,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
97,North York,2.0,Baseball Field,Food Service,Yoga Studio,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [176]:
food_and_amenities = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
food_and_amenities

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,Bar,Yoga Studio,Farmers Market,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
2,Scarborough,3.0,Medical Center,Intersection,Electronics Store,Restaurant,Rental Car Location,Bank,Mexican Restaurant,Breakfast Spot,Discount Store,Diner
3,Scarborough,3.0,Coffee Shop,Pharmacy,Korean Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,Scarborough,3.0,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Gas Station,Diner,Dessert Shop
6,Scarborough,3.0,Department Store,Coffee Shop,Hobby Shop,Chinese Restaurant,Bus Station,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
7,Scarborough,3.0,Bus Line,Bakery,Ice Cream Shop,Park,Metro Station,Yoga Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
8,Scarborough,3.0,American Restaurant,Motel,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
9,Scarborough,3.0,Café,College Stadium,General Entertainment,Skating Rink,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
10,Scarborough,3.0,Indian Restaurant,Light Rail Station,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Gaming Cafe,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
11,Scarborough,3.0,Sandwich Place,Auto Garage,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Bakery,Yoga Studio,Dim Sum Restaurant,Diner,Distribution Center


In [177]:
kind_of_random = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
kind_of_random 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
